<a href="https://colab.research.google.com/github/buketugurlu/LangTutorAI/blob/main/langtutorai3bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Veri Yükleme ve İşleme
file_path = "/content/drive/My Drive/entries.train"
inputs = []  # Hatalı cümleler
targets = []  # Düzeltilmiş cümleler

with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        parts = line.strip().split("\t")  # Satırları tab (\t) ile ayır
        if len(parts) > 5:  # En az 5 sütun varsa
            corrected_label = parts[0]  # Düzeltme etiketi (ilk sütun)
            original_text = parts[4]  # Hatalı cümle (5. sütun)
            if corrected_label == "1":  # Düzeltme varsa
                inputs.append(original_text)
                targets.append(parts[5])  # Düzeltilmiş cümle (6. sütun)

Mounted at /content/drive


In [3]:
# Model ve Tokenizer Yükleme
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [4]:
# Optimizer Tanımlama
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [5]:
# Eğitim Parametreleri
num_epochs = 3
batch_size = 16

In [6]:
# Eğitim Döngüsü
for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(inputs), batch_size):
        input_batch = inputs[i:i+batch_size]
        target_batch = targets[i:i+batch_size]

        # Veriyi Tokenizer İle Kodlama
        inputs_enc = tokenizer(input_batch, padding=True, truncation=True, return_tensors="pt").to(device)
        targets_enc = tokenizer(target_batch, padding=True, truncation=True, return_tensors="pt").to(device)

        # Model Çıktıları ve Kaybı Hesaplama
        outputs = model(input_ids=inputs_enc["input_ids"], labels=targets_enc["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f"Epoch {epoch + 1}, Step {i // batch_size + 1}: Loss = {loss.item()}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Epoch 3, Step 22576: Loss = 0.29517441987991333
Epoch 3, Step 22577: Loss = 0.35770317912101746
Epoch 3, Step 22578: Loss = 0.488254189491272
Epoch 3, Step 22579: Loss = 0.2725049555301666
Epoch 3, Step 22580: Loss = 0.2716313600540161
Epoch 3, Step 22581: Loss = 0.3568219840526581
Epoch 3, Step 22582: Loss = 0.23758617043495178
Epoch 3, Step 22583: Loss = 0.2561863362789154
Epoch 3, Step 22584: Loss = 0.3353123664855957
Epoch 3, Step 22585: Loss = 0.23424723744392395
Epoch 3, Step 22586: Loss = 0.2850981652736664
Epoch 3, Step 22587: Loss = 0.2788282036781311
Epoch 3, Step 22588: Loss = 0.35510966181755066
Epoch 3, Step 22589: Loss = 0.4893946349620819
Epoch 3, Step 22590: Loss = 0.2460593581199646
Epoch 3, Step 22591: Loss = 0.4718018174171448
Epoch 3, Step 22592: Loss = 0.27200910449028015
Epoch 3, Step 22593: Loss = 0.18436001241207123
Epoch 3, Step 22594: Loss = 0.2858182191848755
Epoch 3, Step 22595: Loss = 0.5199289917945862
Epoch 3

In [10]:
# Modeli kaydet
model_dir = "/content/drive/My Drive/bart_model"

model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

print("Model ve tokenizer başarıyla kaydedildi!")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model ve tokenizer başarıyla kaydedildi!


In [12]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Model ve tokenizer'ı yükle
model_dir = "/content/drive/My Drive/bart_model"
model = BartForConditionalGeneration.from_pretrained(model_dir)
tokenizer = BartTokenizer.from_pretrained(model_dir)

# Cihaz seçimi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Test için fonksiyon
def test_model(sentence):
    # Girdi cümlesini tokenleştir
    inputs = tokenizer(sentence, return_tensors="pt", max_length=128, truncation=True).to(device)

    # Model çıktısı
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)

    # Düzeltilmiş cümleyi çöz
    corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_sentence

# Test için örnek cümleler
test_sentences = [
    "I has a apple.",
    "She dont like running in the rain.",
    "He go to the park every day and play football."
]

# Test sonuçları
for sentence in test_sentences:
    print(f"Orijinal Cümle: {sentence}")
    print(f"Düzeltilmiş Cümle: {test_model(sentence)}")
    print("-" * 50)


Orijinal Cümle: I has a apple.
Düzeltilmiş Cümle: I have an apple .
--------------------------------------------------
Orijinal Cümle: She dont like running in the rain.
Düzeltilmiş Cümle: She does n't like running in the rain .
--------------------------------------------------
Orijinal Cümle: He go to the park every day and play football.
Düzeltilmiş Cümle: He goes to the park every day and plays football .
--------------------------------------------------
